In [1]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics import auc, roc_curve, confusion_matrix, classification_report
import fasttext
import certifi
import ssl
import urllib.request
import os
import gzip
import shutil
import pandas as pd
import numpy as np
import subprocess
from sklearn.metrics import accuracy_score,f1_score,balanced_accuracy_score
from sklearn.model_selection import StratifiedKFold,RandomizedSearchCV
from xgboost import XGBClassifier

In [2]:
data = pd.read_excel('/Users/rachael/Downloads/train_df_labelled copy.xlsx')
#the data verified by the third person will be used as the train set finally
file =  data[['content','sentiment']]
file['sentiment'].value_counts()
label_sentiment= file['sentiment'].value_counts()
label_proportions_sentiment= file['sentiment'].value_counts(normalize=True)
print("Counts of each label_sentiment:")
print(label_sentiment)
print("\nProportions of each label_sentiment:")
print(label_proportions_sentiment)

Counts of each label_sentiment:
sentiment
Negative    262
Neither      99
Positive     48
Name: count, dtype: int64

Proportions of each label_sentiment:
sentiment
Negative    0.640587
Neither     0.242054
Positive    0.117359
Name: proportion, dtype: float64


In [3]:
import re
def remove_urls(text):
    # 定义URL的正则表达式
    url_pattern = r'https?://\S+|www\.\S+'
    # 使用re.sub()函数替换URL为空字符串
    no_url_text = re.sub(url_pattern, '', text)
    return no_url_text

def clean_text(text):
    # Remove anything that is not a letter or space
    clean_text = re.sub(r'[^a-zA-Z\s]', '', text)
    # Optional: Convert text to lower case
    clean_text = clean_text.lower()
    return clean_text

In [4]:
#clean the data 
file['content'] = file['content'].apply(remove_urls) 
file['content'] = file['content'].apply(clean_text) 

#convert labels
file['sentiment'] = file['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})

y = file['sentiment']
X = file['content']

/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_5887/195539347.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(remove_urls)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_5887/195539347.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  file['content'] = file['content'].apply(clean_text)
/var/folders/vq/5p2k6x290wzfc5dzz5xm78bm0000gp/T/ipykernel_5887/195539347.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice f

In [5]:
import fasttext

# Path to your FastText pre-trained word embedding file
fasttext_model_path = '/Users/rachael/Downloads/cc.nl.300.bin'

# Load the FastText model
ft = fasttext.load_model(fasttext_model_path)

In [6]:
import spacy
nlp = spacy.load("nl_core_news_sm")
#tokenize
def tokenize(text):
    doc = nlp(text)
    return [token.text for token in doc]

def sentence_to_avg_vec(tokens, model):
    vector_size = model.get_dimension()
    vec = np.zeros(vector_size)
    count = 0
    for token in tokens:
        vec += model.get_word_vector(token)
        count += 1
    if count != 0:
        vec /= count
    return vec

In [7]:
# Initialize parameters
best_score = 0
best_params = {}

#hyperparameter range
param_dist = {
    'learning_rate': [item/100 for item in range(1, 31)],
    'max_depth': [item for item in range(3, 11)],
    'subsample': [item/10 for item in range(5, 11)],
    'colsample_bytree': [item/10 for item in range(5, 11)],
    'lambda': [item/10 for item in range(0, 11)],
    'alpha': [item/10 for item in range(0, 11)],
}

test_scores = []
best_params_list = []
f1_scores = []

# Stratified K-fold for maintaining label distribution, shuffle=True确保每次迭代的数据划分不同
skf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)

In [8]:
# Loop over each fold
for train_index, test_index in skf.split(X, y):
    trainX, testX = X.iloc[train_index], X.iloc[test_index]
    trainy, testy = y.iloc[train_index], y.iloc[test_index]

       # 进行tokenize
    X_train_tokenized = trainX.apply(tokenize)
    X_test_tokenized = testX.apply(tokenize)
    train_vectors = np.array([sentence_to_avg_vec(sentence,ft) for sentence in X_train_tokenized])
    test_vectors = np.array([sentence_to_avg_vec(sentence,ft) for sentence in X_test_tokenized])

    inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
    xgb = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42)
    randomized_search = RandomizedSearchCV(
        estimator=xgb,
        param_distributions=param_dist,
        n_iter=100,  # Number of parameter settings that are sampled
        scoring='f1_macro',
        n_jobs=-1,  # Use all available cores
        cv=inner_cv,
        random_state=42,
        verbose=1  # Set verbosity to get more information during the search
    )

    # Fit Randomized Search
    randomized_search.fit(train_vectors, trainy)

    # Get the best model from Randomized Search
    best_model = randomized_search.best_estimator_
    best_params = randomized_search.best_params_
    best_score = randomized_search.best_score_

    # Evaluate on the test set
    test_predictions = best_model.predict(test_vectors)
    test_score = f1_score(testy,test_predictions, average='macro')

    test_scores.append(test_score)
    best_params_list.append(best_params)

    print(f"Fold test score: {test_score}")
    print(f"Best parameters: {best_params}")

# Summarize the results
mean_test_score = np.mean(test_scores)
std_test_score = np.std(test_scores)

print(f"Mean test score: {mean_test_score}")
print(f"Standard deviation of test scores: {std_test_score}")
print("Best parameters for each fold:")
for i, params in enumerate(best_params_list):
    print(f"Fold {i + 1}: {params}")

Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold test score: 0.351213282247765
Best parameters: {'subsample': 0.7, 'max_depth': 7, 'learning_rate': 0.3, 'lambda': 0.3, 'colsample_bytree': 0.9, 'alpha': 0.2}
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold test score: 0.3767676767676768
Best parameters: {'subsample': 0.8, 'max_depth': 4, 'learning_rate': 0.27, 'lambda': 0.5, 'colsample_bytree': 0.8, 'alpha': 0.3}
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold test score: 0.3822597498904507
Best parameters: {'subsample': 0.5, 'max_depth': 10, 'learning_rate': 0.23, 'lambda': 0.5, 'colsample_bytree': 1.0, 'alpha': 0.0}
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold test score: 0.48125404007756956
Best parameters: {'subsample': 0.5, 'max_depth': 5, 'learning_rate': 0.24, 'lambda': 0.1, 'colsample_bytree': 1.0, 'alpha': 0.2}
Fitting 3 folds for each of 100 candidates, totalling 300 fits
Fold test score: 0.390123

In [9]:
from collections import Counter
best_params_counter = Counter(tuple(sorted(params.items())) for params in best_params_list)
most_common_params = dict(best_params_counter.most_common(1)[0][0])

print("Most common best parameters:", most_common_params)

Most common best parameters: {'alpha': 0.2, 'colsample_bytree': 0.9, 'lambda': 0.3, 'learning_rate': 0.3, 'max_depth': 7, 'subsample': 0.7}


In [7]:
most_common_params = {'subsample': 0.5, 'max_depth': 5, 'learning_rate': 0.24, 'lambda': 0.1, 'colsample_bytree': 1.0, 'alpha': 0.2}


In [8]:
from sklearn.utils.class_weight import compute_class_weight
X_sentences = [item for item in list(X)]
X_tokenized = X.apply(tokenize)
X_embeddings = np.array([sentence_to_avg_vec(sentence,ft) for sentence in X_tokenized])
# Calculate class weights

final_model = XGBClassifier(use_label_encoder=False, eval_metric='mlogloss', random_state=42,**most_common_params)
final_model.fit(X_embeddings, y)

XGBClassifier(alpha=0.2, base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=1.0, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric='mlogloss',
              feature_types=None, gamma=None, grow_policy=None,
              importance_type=None, interaction_constraints=None, lambda=0.1,
              learning_rate=0.24, max_bin=None, max_cat_threshold=None,
              max_cat_to_onehot=None, max_delta_step=None, max_depth=5,
              max_leaves=None, min_child_weight=None, missing=nan,
              monotone_constraints=None, multi_strategy=None, n_estimators=None,
              n_jobs=None, ...)

In [9]:
test = pd.read_excel('/Users/rachael/Downloads/test_df_labelled.xlsx')
#the data verified by the third person will be used as the train set finally
test = test[['content','sentiment']]
test['content'] = test['content'].apply(remove_urls) 
test['content'] = test['content'].apply(clean_text) 
test['sentiment'] = test['sentiment'].replace({'Positive':0,'Negative': 1, 'Neither': 2})
test_tokenized = test['content'].apply(tokenize)
test_embedding = np.array([sentence_to_avg_vec(sentence,ft) for sentence in test_tokenized])
test_predictions = final_model.predict(test_embedding)
test_score = accuracy_score(test['sentiment'], test_predictions)
conf_matrix = confusion_matrix(test['sentiment'], test_predictions)
class_report = classification_report(test['sentiment'], test_predictions)
print(class_report)

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        38
           1       0.64      0.91      0.75       245
           2       0.63      0.27      0.38       124

    accuracy                           0.63       407
   macro avg       0.42      0.39      0.37       407
weighted avg       0.58      0.63      0.56       407

